In [1]:
!pip install datasets
!pip install vllm jsonlines tqdm sacrebleu datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 10.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

import os
import json
from transformers import AutoTokenizer
import sacrebleu

from vllm import LLM, SamplingParams

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import sacrebleu

INFO 03-22 15:15:13 [__init__.py:256] Automatically detected platform cuda.


In [3]:
dataset = load_dataset(
    "WeixiangYan/CodeTransOcean",
    data_files={"test": "CodeTrans Datasets/MultilingualTrans/multilingual_test.json"},
    split="test"
)

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

multilingual_test.json:   0%|          | 0.00/44.5M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
languages = ["C", "C#", "C++", "Go", "Java", "Python", "VB", "PHP"]

# 言語ペアごとのデータセット作成
pair_datasets = {}
for src_lang in languages:
    for tgt_lang in languages:
        if src_lang == tgt_lang:
            continue

        # 特定の言語ペアのみを含むサンプルをフィルタリング
        def has_language_pair(sample):
            return (src_lang in sample and sample[src_lang] is not None and sample[src_lang] != "" and
                    tgt_lang in sample and sample[tgt_lang] is not None and sample[tgt_lang] != "")

        filtered_dataset = dataset.filter(has_language_pair)
        pair_datasets[(src_lang, tgt_lang)] = filtered_dataset

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15090 [00:00<?, ? examples/s]

In [ ]:
os.environ["HF_TOKEN"] = "WRITE ME Your Token"  # 必要なら適宜変更

base_model_id = "kazuyamaa/gemma-2-2b-code-translate-dpo-merged"
llm = LLM(
    model=base_model_id,
    tensor_parallel_size=1,      # 使用するGPU数（必要に応じて調整）
    trust_remote_code=True,
    download_dir="./model_cache",# キャッシュディレクトリ
    dtype="half",                # 半精度を使用
    gpu_memory_utilization=0.95,   # GPUメモリ使用率を増加
    max_model_len=8192         # 最大モデル長（調整可能）
)

config.json:   0%|          | 0.00/874 [00:00<?, ?B/s]

WARNING 03-22 17:10:31 [config.py:2599] Casting torch.bfloat16 to torch.float16.
INFO 03-22 17:10:31 [config.py:583] This model supports multiple tasks: {'embed', 'score', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 03-22 17:10:31 [config.py:1693] Chunked prefill is enabled with max_num_batched_tokens=8192.


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

INFO 03-22 17:10:36 [core.py:53] Initializing a V1 LLM engine (v0.8.1) with config: model='kazuyamaa/gemma-2-2b-code-translate-dpo-merged', speculative_config=None, tokenizer='kazuyamaa/gemma-2-2b-code-translate-dpo-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir='./model_cache', load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=None, served_model_name=kazuyamaa/gemma-2-2b-code-translate-dpo-merged, num_scheduler_steps=1, multi_step_stream_outputs=

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


WARNING 03-22 17:10:36 [utils.py:2282] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x7eacacfc8d90>
INFO 03-22 17:10:37 [parallel_state.py:967] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 03-22 17:10:37 [cuda.py:215] Using Flash Attention backend on V1 engine.
INFO 03-22 17:10:37 [gpu_model_runner.py:1164] Starting to load model kazuyamaa/gemma-2-2b-code-translate-dpo-merged...
WARNING 03-22 17:10:37 [config.py:3670] `torch.compile` is turned on, but the model kazuyamaa/gemma-2-2b-code-translate-dpo-merged does not support it. Please open an issue on GitHub if you want it to be supported.
ERROR 03-22 17:10:37 [core.py:340] EngineCore hit an exception: Traceback (most recent call last):
ERROR 03-22 17:10:37 [core.py:340]   File "/usr/local/lib/python3.10/dist-packages/vllm/v1/engine/core.py", line 332, in run_engine_core
ERROR 03-22 17:10:37 [

In [ ]:
# -------------------------------
# 3. サンプリングパラメータの設定と各言語ペアでの推論
# -------------------------------
sampling_params = SamplingParams(
    temperature=0.0,      # 温度0で決定的な出力
    max_tokens=4096,       # 最大出力トークン数（必要に応じて調整）
    repetition_penalty=1.2
)

# 各言語ペアごとの生成結果と正解コードを格納する辞書
results = {}


# 各言語ペアごとにバッチ推論を実施
for src_lang in languages:
    for tgt_lang in languages:
        if src_lang == tgt_lang:
            continue

        # この言語ペアのデータセットを取得
        pair_data = pair_datasets.get((src_lang, tgt_lang), [])

        if len(pair_data) == 0:
            print(f"スキップ: {src_lang} -> {tgt_lang} (サンプルなし)")
            continue

        prompts = []
        ref_codes = []

        # 各サンプルからプロンプトを準備（最大N個までに制限）
        max_samples = min(len(pair_data), 10)  # 例として10個に制限

        for i in range(max_samples):
            sample = pair_data[i]
            source_code = sample[src_lang]
            prompt = f"{src_lang} to {tgt_lang}: {source_code}"
            prompts.append(prompt)
            ref_codes.append(sample[tgt_lang])

        # サンプル数を出力
        print(f"処理中: {src_lang} -> {tgt_lang} ({len(prompts)} サンプル)")

        # バッチサイズが小さい場合、処理が実行されない問題を回避するためのチェック
        if len(prompts) == 0:
            print(f"スキップ: {src_lang} -> {tgt_lang} (サンプルなし)")
            continue

        # vLLMによるバッチ生成
        outputs = llm.generate(prompts, sampling_params)

        generated_codes = []
        for output in outputs:
            generated_text = output.outputs[0].text
            generated_codes.append(generated_text)

        # 結果を保存
        results[(src_lang, tgt_lang)] = {
            "generated": generated_codes,
            "reference": ref_codes
        }

        print(f"完了: {src_lang} -> {tgt_lang}")

In [ ]:

# -------------------------------
# 4. BLEUスコアの計算
# -------------------------------
bleu_scores = {}
for (src, tgt), texts in results.items():
    # sacreBLEUはリファレンスをリストのリスト形式で受け取るのでラップする
    refs = [[ref] for ref in texts["reference"]]
    bleu = sacrebleu.corpus_bleu(texts["generated"], list(zip(*refs)))
    bleu_scores[(src, tgt)] = bleu.score
    print(f"BLEU {src} -> {tgt}: {bleu.score:.2f}")

# オプション: 結果をファイルに保存
with open("results_vllm.jsonl", "w", encoding="utf-8") as f:
    for (src, tgt), data in results.items():
        out = {
            "src_lang": src,
            "tgt_lang": tgt,
            "generated": data["generated"],
            "reference": data["reference"],
            "bleu": bleu_scores[(src, tgt)]
        }
        f.write(json.dumps(out, ensure_ascii=False) + "\n")


In [8]:
!pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 96.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 121.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# 言語数
n = len(languages)

# BLEUスコア用の行列を作成（初期値は NaN）
bleu_matrix = np.empty((n, n))
bleu_matrix[:] = np.nan

# 辞書から各言語ペアのBLEUスコアを行列に配置
for (src, tgt), score in bleu_scores.items():
    if src not in languages or tgt not in languages:
        continue  # languages リストにない言語はスキップする
    i = languages.index(src)
    j = languages.index(tgt)
    bleu_matrix[i, j] = score

# プロット設定
plt.figure(figsize=(10, 8))
# vmin, vmax はBLEUスコアの範囲に合わせて調整可能
im = plt.imshow(bleu_matrix, cmap='viridis', vmin=0, vmax=100)
plt.colorbar(im)

# 軸ラベルの設定
plt.xticks(np.arange(n), languages)
plt.yticks(np.arange(n), languages)
plt.xlabel("Target Language")
plt.ylabel("Source Language")
plt.title("BLEU Scores for Programming Language Translation")

# 各セルにBLEUスコアを表示（NaNのセルはスキップ）
for i in range(n):
    for j in range(n):
        if not np.isnan(bleu_matrix[i, j]):
            plt.text(j, i, f"{bleu_matrix[i, j]:.2f}", ha='center', va='center', color='white')

plt.show()